# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Get the path and file name pattern from the simulation file naming convention
events_fname = Path(sim.env.events_log_fname)

# Get the path to the file which is one level up from the logging outputs
fpath = events_fname.parents[1]

# Get the filename and update it to the correct name
fname = events_fname.name
fname = fname.replace("_events.log", "_metrics_inputs.yaml")

# Load the metrics data
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 95.6%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.98,0.98,0.98,0.98,0.88,0.86,0.98,0.98,0.98,0.88,0.98,0.96,0.91,0.98,0.98,0.98,0.98,0.98,0.98,0.92,0.98,0.98,0.98,0.93,0.94,0.95,0.86,0.92,0.98,0.98,0.98,0.92,0.98,0.93,0.82,0.98,0.98,0.98,0.89,0.98,0.98,0.86,0.98,0.98,0.94,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.95,0.97,0.98,0.76,0.98,0.98,0.96,0.98,0.98,0.97,0.98,0.98,0.81,0.97,0.98,0.98,0.88,0.98,0.91,0.98,0.98,0.98,0.98,0.98


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.98
2004,0.96
2005,0.97
2006,0.95
2007,0.93
2008,0.97
2009,0.98
2010,0.92
2011,0.93


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.95
3,0.95
4,0.94
5,0.95
6,0.95
7,0.96
8,0.96
9,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.99
     3          0.99
     4          0.98
     5          0.99
     6          0.99
     7          1.00
     8          0.99
     9          0.98
     10         0.97
     11         0.97
     12         0.95
2004 1          0.95
     2          0.95
     3          0.94
     4          0.94
     5          0.95
     6          0.96
     7          0.96
     8          0.95
     9          0.96
     10         0.98
     11         0.98
     12         0.97

## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.46,0.47,0.47,0.47,0.42,0.41,0.47,0.47,0.47,0.42,0.47,0.46,0.43,0.47,0.47,0.47,0.47,0.46,0.47,0.43,0.47,0.47,0.47,0.44,0.45,0.45,0.42,0.43,0.47,0.47,0.47,0.43,0.47,0.44,0.39,0.46,0.47,0.47,0.42,0.47,0.46,0.41,0.47,0.47,0.45,0.47,0.47,0.47,0.47,0.46,0.47,0.47,0.47,0.47,0.47,0.47,0.45,0.46,0.47,0.36,0.47,0.47,0.46,0.47,0.47,0.47,0.47,0.47,0.39,0.46,0.47,0.47,0.42,0.47,0.44,0.47,0.47,0.47,0.47,0.47


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.45
2005,0.48
2006,0.44
2007,0.48
2008,0.50
2009,0.46
2010,0.39
2011,0.47


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.50
3,0.50
4,0.40
5,0.38
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.63
     2          0.37
     3          0.38
     4          0.53
     5          0.35
     6          0.31
     7          0.30
     8          0.30
     9          0.27
     10         0.43
     11         0.54
     12         0.54
2004 1          0.53
     2          0.50
     3          0.50
     4          0.38
     5          0.29
     6          0.38
     7          0.27
     8          0.40
     9          0.57
     10         0.56
     11         0.49
     12         0.49

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.98
2004,0.99
2005,1.02
2006,0.99
2007,1.00
2008,1.02
2009,1.00
2010,0.99
2011,1.01


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.92
2,0.99
3,1.03
4,1.00
5,1.03
6,1.05
7,1.01
8,1.00
9,1.02


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.86
     2                 1.11
     3                 0.91
     4                 1.10
     5                 0.95
     6                 1.07
     7                 0.97
     8                 0.99
     9                 0.79
     10                1.25
     11                0.98
     12                0.79
2004 1                 0.60
     2                 0.97
     3                 1.37
     4                 1.11
     5                 1.11
     6                 1.08
     7                 1.15
     8                 0.95
     9                 1.09
     10                0.87
     11                0.99
     12                0.81

## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $498,193.10/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","2,660,000.00","97,726,562.50"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"10,433,781.25"
2004,"13,031,250.00"
2005,"10,432,250.00"
2006,"11,461,937.50"
2007,"13,232,250.00"
2008,"10,490,625.00"
2009,"12,343,187.50"
2010,"12,049,437.50"
2011,"12,463,500.00"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,083.33","543,010.42","543,083.33",0.00,0.00
2,"496,052.08","496,125.00","496,052.08",0.00,0.00
3,"541,989.58","541,989.58","541,625.00",0.00,0.00
4,"524,635.42","524,635.42","525,000.00",0.00,0.00
5,"542,500.00","542,500.00","542,500.00",0.00,"31,596,875.00"
6,"525,000.00","525,000.00","525,000.00","2,660,000.00",0.00
7,"542,500.00","542,500.00","542,500.00",0.00,"31,982,812.50"
8,"542,500.00","542,500.00","542,500.00",0.00,0.00
9,"525,000.00","525,000.00","525,000.00",0.00,"34,146,875.00"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       2,750,000.00  
     6                   52,500.00            266,000.00               0.00  
     7                   54,250.00                  0.00       2,750,000.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00       2,750,000.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00  
2004 1                   54,250.00                  0.00               0.00  
     2                   50,750.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       3,373,437.50  
     6                   52,500.00            266,000.00               0.00  
     7                   54,250.00                  0.00       3,218,750.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00       4,251,562.50  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.78,0.78,0.79,0.25,0.50


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.62,0.63,0.64,0.10,0.13
2004,0.93,0.93,0.92,0.34,0.62
2005,0.88,0.86,0.85,0.38,0.37
2006,0.82,0.82,0.81,0.28,0.52
2007,0.84,0.86,0.87,0.17,0.60
2008,0.78,0.77,0.76,0.10,0.58
2009,0.75,0.77,0.76,0.24,0.33
2010,0.73,0.71,0.75,0.17,0.64
2011,0.74,0.76,0.75,0.34,0.52


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,771,908.86"
Repair,"19,084,598.96"
Crew Transfer,"1,496,473.97"
Site Travel,0.00
Mobilization,"15,000,000.00"
Weather Delay,"29,206,510.42"
No Requests,"23,192,752.60"
Not in Shift,"28,814,098.96"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,771,908.86","2,771,908.86"
Repair,0.00,0.00,0.00,"19,084,598.96","19,084,598.96"
Crew Transfer,0.00,0.00,0.00,"1,496,473.97","1,496,473.97"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"15,000,000.00","15,000,000.00"
Weather Delay,0.00,0.00,0.00,"29,206,510.42","29,206,510.42"
No Requests,0.00,0.00,0.00,"23,192,752.60","23,192,752.60"
Not in Shift,0.00,0.00,0.00,"28,814,098.96","28,814,098.96"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      65,351.56
     Repair          612,812.50
     Crew Transfer    56,213.54
     Site Travel           0.00
     Mobilization  1,500,000.00
     Weather Delay   579,437.50
     No Requests   6,983,770.83
     Not in Shift    636,195.31
2004 Maintenance     457,971.35
     Repair        2,583,119.79

In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                   
1     Maintenance    196,182.29
      Repair         220,335.94
      Crew Transfer   41,398.44
      Site Travel          0.00
      Mobilization         0.00
      Weather Delay  202,380.21
      No Requests    540,713.54
      Not in Shift   428,166.67
2     Maintenance    188,106.77
      Repair         204,440.10

In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          20,416.67
           Crew Transfer    2,406.25
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   17,500.00
           No Requests    117,177.08
           Not in Shift     6,781.25
     2     Maintenance          0.00
           Repair          28,054.69
           Crew Transfer    3,244.79
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   15,531.25
           No Requests     84,091.15
           Not in Shift    16,078.12
     3     Maintenance          0.00
           Repair          25,994.79
           Crew Transfer    2,880.21
           Site Travel          0.00

## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
generator,"117,399,809.91"


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
generator delay                  0.00              0.00   57,740,328.12   
          maintenance            0.00              0.00    2,771,908.86   
          repair                 0.00              0.00   19,084,598.96   

                         total_cost  
component action                     
generator delay       57,740,328.12  
          maintenance  2,771,908.86  
          repair      19,084,598.96

In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,generator,"1,226,500.00",0.00,"1,942,354.17","3,168,854.17"
2004,generator,"5,432,000.00",0.00,"11,344,208.33","16,776,208.33"
2005,generator,"3,481,000.00",0.00,"4,422,635.42","7,903,635.42"
2006,generator,"3,980,000.00",0.00,"7,740,713.54","11,720,713.54"
2007,generator,"4,618,500.00",0.00,"11,169,072.92","15,787,572.92"
2008,generator,"3,490,500.00",0.00,"7,083,570.31","10,574,070.31"
2009,generator,"2,560,000.00",0.00,"7,047,119.79","9,607,119.79"
2010,generator,"4,019,000.00",0.00,"9,422,067.71","13,441,067.71"
2011,generator,"3,647,000.00",0.00,"9,875,190.11","13,522,190.11"


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,generator,"1,240,000.00",0.00,"1,080,807.29","2,320,807.29"
2,generator,"1,261,500.00",0.00,"983,171.88","2,244,671.88"
3,generator,"1,457,500.00",0.00,"1,056,453.13","2,513,953.13"
4,generator,"1,620,500.00",0.00,"1,064,601.56","2,685,101.56"
5,generator,"8,053,000.00",0.00,"23,751,747.40","31,804,747.40"
6,generator,"4,156,000.00",0.00,"1,899,380.21","6,055,380.21"
7,generator,"7,022,500.00",0.00,"20,951,606.77","27,974,106.77"
8,generator,"1,694,500.00",0.00,"1,067,208.33","2,761,708.33"
9,generator,"6,225,000.00",0.00,"26,342,739.58","32,567,739.58"


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     generator       22,000.00              0.00       44,697.92   
     2     generator       45,500.00              0.00       61,158.85   
     3     generator       91,000.00              0.00       46,739.58   
     4     generator       44,500.00              0.00       41,544.27   
     5     generator      390,500.00              0.00    1,260,093.75   
     6     generator      174,500.00              0.00       70,015.63   
     7     generator       33,500.00              0.00       27,671.88   
     8     generator       37,500.00              0.00       39,703.13   
     9     generator       34,500.00              0.00       60,010.42   
     10    generator      119,500.00              0.00      105,856.77   
     11    generator      107,500.00              0.00       53,648.44   
     12    generator      126,000.00              0.00      131,213.54   
2004 1     generator      201,000.00              0.00      162,750.00   
     2     generator      206,000.00              0.00      152,250.00   
     3     generator      282,000.00              0.00      161,875.00   
     4     generator      258,500.00              0.00      155,750.00   
     5     generator    1,253,500.00              0.00    3,035,312.50   
     6     generator      578,000.00              0.00      246,958.33   
     7     generator      949,000.00              0.00    2,879,750.00   
     8     generator      241,000.00              0.00      161,000.00   
     9     generator      896,500.00              0.00    3,908,187.50   
     10    generator      201,000.00              0.00      161,000.00   
     11    generator      191,500.00              0.00      156,625.00   
     12    generator      174,000.00              0.00      162,750.00   
2005 1     generator      125,000.00              0.00      161,000.00   
     2     generator      168,000.00              0.00      145,250.00   
     3     generator      183,500.00              0.00      151,156.25   
     4     generator      189,500.00              0.00      125,088.54   

                       total_cost  
year month component               
2003 1     generator    66,697.92  
     2     generator   106,658.85  
     3     generator   137,739.58  
     4     generator    86,044.27  
     5     generator 1,650,593.75  
     6     generator   244,515.63  
     7     generator    61,171.88  
     8     generator    77,203.13  
     9     generator    94,510.42  
     10    generator   225,356.77  
     11    generator   161,148.44  
     12    generator   257,213.54  
2004 1     generator   363,750.00  
     2     generator   358,250.00  
     3     generator   443,875.00  
     4     generator   414,250.00  
     5     generator 4,288,812.50  
     6     generator   824,958.33  
     7     generator 3,828,750.00  
     8     generator   402,000.00  
     9     generator 4,804,687.50  
     10    generator   362,000.00  
     11    generator   348,125.00  
     12    generator   336,750.00  
2005 1     generator   286,000.00  
     2     generator   313,250.00  
     3     generator   334,656.25  
     4     generator   314,588.54

## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"368,894.27","91,480.50","81,937.50",637.00
major repair,"79,757.02","1,587.00","1,117.00",28.00
major replacement,"192,343.87","10,876.50","192,343.87",51.00
manual reset,"248,666.64","37,915.00","12,479.75","5,637.00"
medium repair,"12,964.64","9,318.00","6,841.50",195.00
minor repair,"85,130.62","32,536.75","13,869.50","2,242.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,585,187,528.00"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,585,187,528.00","122,268,191.50","123,748,910.00","122,841,299.50","123,458,301.00","109,765,549.00","107,899,930.00","122,790,385.00","123,146,491.50","122,431,699.00","110,565,768.00","122,795,743.50","120,100,322.50","112,633,266.50","123,533,158.00","122,894,419.00","123,178,759.50","123,226,926.50","122,134,800.50","123,151,733.50","113,528,565.00","123,632,722.50","122,959,343.00","123,129,840.50","116,493,846.50","119,330,002.00","119,637,808.00","109,152,957.50","113,116,805.50","123,239,286.00","123,400,797.50","123,130,491.00","114,100,942.00","123,145,333.00","116,515,937.00","101,804,245.50","122,241,630.50","122,759,667.50","123,035,458.00","111,190,918.00","123,005,586.00","121,877,684.00","108,323,574.00","123,619,813.00","122,829,346.50","117,543,400.50","122,661,579.50","122,900,154.00","123,220,328.50","123,454,360.00","122,036,184.00","122,613,196.00","122,766,898.00","122,763,935.00","122,988,041.50","122,635,186.50","123,080,446.50","119,420,410.50","121,844,628.00","123,109,520.50","94,120,055.50","122,363,551.50","122,852,920.00","120,238,423.50","122,563,875.50","122,982,401.50","122,554,849.00","122,863,230.00","122,637,352.00","101,777,083.50","121,482,110.50","122,765,711.00","122,471,616.00","110,519,119.00","122,971,053.50","114,770,049.00","122,996,145.50","123,457,231.50","122,590,176.00","122,706,013.50","122,698,037.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"868,588,613.00"
2004,"939,033,473.50"
2005,"1,000,627,511.50"
2006,"932,859,692.00"
2007,"999,779,495.00"
2008,"1,063,463,815.00"
2009,"973,848,255.50"
2010,"826,018,522.00"
2011,"990,592,743.00"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,069,700,572.00"
2,"811,413,751.00"
3,"887,231,511.00"
4,"694,429,865.50"
5,"680,117,526.00"
6,"552,527,846.00"
7,"584,941,462.00"
8,"637,531,224.00"
9,"791,997,281.50"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     112,254,623.00
     2      59,237,874.00
     3      67,520,133.50
     4      91,775,562.00
     5      62,206,048.50
     6      53,806,611.00
     7      54,022,818.50
     8      53,739,094.00
     9      47,484,468.50
     10     76,288,618.50
     11     94,042,746.00
     12     96,210,015.50
2004 1      95,207,293.00
     2      84,068,754.50
     3      88,802,194.00
     4      65,331,737.50
     5      51,252,105.50
     6      65,313,566.50
     7      47,774,358.00
     8      71,356,754.50
     9      98,769,642.50
     10     99,731,063.50
     11     84,180,086.00
     12     87,245,918.00

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [51]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [53]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [54]:
SAM_settings = "SAM_Singleowner_defaults.yaml"
metrics.sam_settings = load_yaml(sim.env.data_dir / "windfarm", SAM_settings)
metrics._setup_pysam()

In [55]:
metrics.pysam_all_outputs()

,Value
Metric,
After Tax NPV ($),"33,705,182.41"
Nominal LCOE ($/kW),-0.01
Real LCOE ($/kW),-0.00
After Tax IRR (%),14.59


In [56]:
sim.env.cleanup_log_files(log_only=False)